In [31]:
#####Marcov Switching model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(98537)

In [32]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [33]:
####データの発生####
##データの設定
s = 4   #切り替え数
k1 = 2   #観測モデルの応答変数のパラメータ数
k2 = 2   #観測モデルの説明変数のパラメータ数
n = 3000   #サンプル数

In [34]:
##推移確率のパラメータを設定
#初期確率の設定
gamma = np.repeat(1/s, s)
gammat = gamma

#マルコフ推移行列を設定
theta1 = np.array([0.7, 0.3, 0, 0])
theta2 = np.array([0, 0, 0.6, 0.4])
theta3 = np.array([0.75, 0.25, 0, 0])
theta4 = np.array([0, 0, 0.8, 0.2])
theta = np.vstack((theta1, theta2, theta3, theta4))
thetat = theta

##観測モデルのパラメータを設定
#標準化価格のパラメータを設定
k11 = 0
tau1 = np.array([8.0, 7.0, 9.0, 7.5])
tau2 = np.array([0.5, 3.5, 1.0, 2.25])
tau = np.hstack((tau1, tau2)).reshape(s, k1, order="F")
taut1 = tau1; taut2 = tau2; taut = tau

#売上点数のパラメータ
k12 = 1
beta0 = np.array([2.1, 3.3, 1.5, 2.7])
beta1 = np.array([1.0, 1.8, 0.8, 1.3])
beta2 = np.array([0.5, 0.7, 0.3, 0.6])
beta = np.vstack((beta0, beta1, beta2))
betat0 = beta0; betat1 = beta1; betat2 = beta2; betat = beta
Sigma = 0.5
Sigmat = Sigma

In [35]:
##応答変数の発生
#データの保存用配列
Z = np.zeros((n, s), dtype="int")
z = np.zeros((n), dtype="int")
y = np.zeros((n, k1))
X = np.hstack((np.repeat(1, n).reshape(n, 1), np.zeros((n, k2))))
X[:, k2] = np.random.binomial(1, 0.4, n)

In [36]:
#1期目のデータを生成
Z[0, ] = np.random.multinomial(1, gamma, 1)
seg = np.dot(Z[0], np.arange(s)); z[0] = seg
y[0, k11] = np.random.beta(tau1[seg], tau2[seg], 1)   #価格を生成
X[0, k12] = y[0, k11]
y[0, k12] = np.dot(X[0, ], beta[:, seg]) + np.random.normal(0, Sigma, 1)   #売上点数を生成

In [37]:
#期間ごとに応答変数を生成
for i in range(1, n):
    #マルコフ推移行列から潜在変数を生成
    old_seg = z[i-1]
    Z[i, ] = np.random.multinomial(1, theta[old_seg, ], 1)
    seg = np.dot(Z[i, ], np.arange(s)); z[i] = seg

    #潜在変数から観測データを生成
    y[i, k11] = np.random.beta(tau1[seg], tau2[seg], 1)   #価格を生成
    X[i, k12] = y[i, k11]
    y[i, k12] = np.dot(X[i, ], beta[:, seg]) + np.random.normal(0, Sigma, 1)   #売上点数を生成

In [38]:
#データの確認
[print(np.mean(y[z==i, ], axis=0)) for i in range(s)]
print(np.mean(Z, axis=0))

[0.9412971  3.25670548]
[0.65639778 4.75657832]
[0.90157827 2.32755216]
[0.76562276 3.96635219]
[0.49266667 0.202      0.202      0.10333333]


In [39]:
####マルコフ連鎖モンテカルロ法でMarcov Switching modelを推定####
##対数ベータ事後分布の和
def LLho(tau_log, beta_par, tau_inv, y_vec, n, index):
    #パラメータの設定
    alpha = np.exp(tau_log)

    #ベータ分布の対数尤度
    LL_beta = np.sum(scipy.stats.beta.logpdf(y_vec, alpha[0], alpha[1]))

    #多変量正規分布の対数事前分布
    er = tau_log - beta_par
    LL_mvn = -1/2 * np.dot(np.dot(er, tau_inv), er)

    #対数事後分布の和
    LL = LL_beta + LL_mvn
    return LL

In [40]:
##対数ベータ事後分布の勾配ベクトルを設定
def dbeta(tau_log, beta_par, tau_inv, y_vec, n, index):
    #パラメータの設定
    alpha = np.exp(tau_log)

    #ベータ分布の勾配ベクトル
    const = n*scipy.special.digamma(np.sum(alpha))
    g1 = const - n*scipy.special.digamma(alpha[0]) + np.sum(np.log(y_vec))
    g2 = const - n*scipy.special.digamma(alpha[1]) + np.sum(np.log(1-y_vec))
    g = np.array([g1, g2])
    
    #対数事前分布の勾配ベクトル
    er = tau_log - beta_par
    dmvn = np.dot(tau_inv, er.T)   
    
    #対数事後分布の勾配ベクトルの和
    LLd = -(g + dmvn)
    return LLd

In [41]:
##リープフロッグ法を解く関数
def leapfrog(r, z1, D, e, L): 
    def leapfrog_step(r, z1, e):
        r2 = r - e * D(z1, beta_par, tau_inv, y_vec, m, index) / 2
        z2 = z1 + e * r2
        r2 = r2 - e * D(z2, beta_par, tau_inv, y_vec, m, index) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z1]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [42]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 3000
keep = 2
iter = 0
burnin = int(2000/keep)
disp = 100
e = 0.0005
L = 5

In [43]:
##事前分布の設定
#マルコフ推移行列の事前分布
alpha0 = 0.1
s0 = 0.5
v0 = 0.5

#回帰ベクトルの事前分布
mu_par = np.repeat(0, k2+1)
Cov_inv = np.linalg.inv(np.diag(np.repeat(100, k2+1)))

#ベータ分布の事前分布
beta_par = np.repeat(0, 2)
tau_inv = np.linalg.inv(np.diag(np.repeat(100, 2)))

In [44]:
##真値の設定
#潜在変数の真値
Zi = Z
z_vec = np.dot(Zi, np.arange(s))

#推移行列の真値
gamma = gammat
theta = thetat

#モデルパラメータの真値
beta = betat
Sigma = Sigmat
tau = taut

In [56]:
##初期値の設定
#推移行列の初期値
gamma = np.repeat(1/s, s)
theta1 = np.array([0.6, 0.4, 0, 0])
theta2 = np.array([0, 0, 0.6, 0.4])
theta3 = np.array([0.6, 0.4, 0, 0])
theta4 = np.array([0, 0, 0.6, 0.4])
theta = np.vstack((theta1, theta2, theta3, theta4))

#潜在変数の初期値
Zi = np.random.multinomial(1, np.repeat(1/s, s), n)
z_vec = np.dot(Zi, np.arange(s))

#モデルパラメータの初期値
beta = np.zeros((X.shape[1], s))
tau1 = np.zeros((s)); tau2 = np.zeros((s))
for j in range(s):
    index = np.arange(n)[z_vec==j]
    tau1[j] = np.mean(y[index, k11]) * np.power(s, 2); tau2[j] = (1 - np.mean(y[index, k11])) * np.power(s, 2)
beta = np.repeat(np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y[:, k12]), s).reshape(k2+1, s)
beta[0, ] = np.array([1.5, 2.5, 1, 2])
Sigma = np.std(y[:, k12] - np.sum(X * beta.T[z_vec, ], axis=1))

In [57]:
##インデックスを設定
index_s = [j for j in range(s)]
for j in range(s):
    index_s[j] = np.arange(s)[theta[j, ] > 0]
index_t21 = np.arange(n-1)
index_t22 = np.arange(1, n)

In [58]:
##サンプリング結果の格納用配列
BETA = np.zeros((k2+1, s, int(R/keep)))
SIGMA = np.zeros((int(R/keep)))
TAU = np.zeros((s, 2, int(R/keep)))
THETA = np.zeros((s, s, int(R/keep)))
SEG = np.zeros((n, s, int(R/keep)))

In [59]:
##対数尤度の基準値
#真値の対数尤度
seg = np.dot(Z, np.arange(s))
LLbest1 = np.sum(scipy.stats.norm.logpdf(y[:, k12], np.sum(X * betat.T[seg, ], axis=1), Sigmat))
LLbest2 = np.sum(scipy.stats.beta.logpdf(y[:, k11], taut1[seg], taut2[seg]))
LLbest = LLbest1 + LLbest2
print(LLbest)

2280.3830613154287


In [60]:
####HMCでパラメータをサンプリング####
for rp in range(R):
    
    ##レコードごとの尤度と混合率を設定
    #潜在変数を条件づけた尤度
    Lho1 = np.zeros((n, s)); Lho2 = np.zeros((n, s))
    for j in range(s):
        mu = np.dot(X, beta[:, j])
        Lho1[:, j] = scipy.stats.beta.pdf(y[:, k11], tau1[j], tau2[j])
        Lho2[:, j] = scipy.stats.norm.pdf(y[:, k12], mu, Sigma)

    #混合率を設定
    theta_dt1 = np.zeros((n, s)); theta_dt2 = np.zeros((n, s))
    theta_dt1[0, ] = gamma
    theta_dt1[index_t22, ] = theta[z_vec[index_t21], ]
    theta_dt2[index_t21, ] = theta.T[z_vec[index_t22], ]

    ##多項分布から潜在変数をサンプリング
    #潜在変数の割当確率
    Li = theta_dt1 * Lho1 * Lho2   #結合分布
    Prob = Li / np.dot(Li, np.repeat(1, s)).reshape(n, 1)

    #潜在変数をサンプリング
    Zi = np.array(rmnom(Prob, n, s, np.arange(n)).todense())
    z_vec = np.dot(Zi, np.arange(s))


    ##混合率をサンプリング
    #潜在変数間の共起を設定
    rf = np.dot(Zi[index_t21, ].T, Zi[index_t22, ])
    gamma = np.random.dirichlet(np.sum(Zi, axis=0) + alpha0, 1).reshape(-1)
    
    for j in range(s):
        #ベータ分布のパラメータ
        rf_s = rf[j, index_s[j]]
        s1 = rf_s[0] + s0
        v1 = rf_s[1] + v0

        #ベータ分布からパラメータをサンプリング
        par = np.random.beta(s1, v1, 1)
        theta[j, index_s[j]] = np.append(par, 1-par)
    

    ##回帰ベクトルをサンプリング
    #インデックスを設定
    beta = np.zeros((k2+1, s))
    index_z = [j for j in range(s)]
    for j in range(s): 
        index_z[j] = np.where(Zi[:, j]==1)[0]

        #事後分布のパラメータ
        x = X[index_z[j], ]
        Xy = np.dot(x.T, y[index_z[j], k12])
        inv_XXV = np.linalg.inv(np.dot(x.T, x) + Cov_inv)
        beta_mu = np.dot(inv_XXV, Xy + np.dot(Cov_inv, mu_par))   #回帰ベクトルの期待値

        #多変量正規分布からパラメータをサンプリング
        beta[:, j] = np.random.multivariate_normal(beta_mu, np.power(Sigma, 2)*inv_XXV, 1).reshape(-1)

    ##標準偏差をサンプリング
    #逆ガンマ分布のパラメータ
    er = y[:, k12] - np.sum(X * beta.T[z_vec, ], axis=1)
    s1 = np.dot(er.T, er) + s0
    v1 = n + v0

    #逆ガンマ分布からパラメータをサンプリング
    Sigma = np.sqrt(1/np.random.gamma(v1/2, 1/(s1/2), 1))


    ##HMCでベータ分布のパラメータをサンプリング
    accept_prob = np.repeat(0, s)
    tau = np.zeros((s, 2))
    
    #潜在変数ごとにパラメータをサンプリング
    for j in range(s):
        #データの設定
        index = index_z[j]
        m = index.shape[0]
        y_vec = y[index, k11]

        #HMCの新しいパラメータを生成
        rold = np.random.normal(0, 1, 2)
        tau_logd = np.log(np.append(tau1[j], tau2[j]))

        #リープフロッグ法による1ステップ移動
        res = leapfrog(rold, tau_logd, dbeta, e, L)
        rnew = res[0]
        tau_logn = res[1]

        #移動前と移動後のハミルトニアン
        Hnew = -LLho(tau_logn, beta_par, tau_inv, y_vec, m, index) + np.sum(np.power(rnew, 2)) / 2
        Hold = -LLho(tau_logd, beta_par, tau_inv, y_vec, m, index) + np.sum(np.power(rold, 2)) / 2

        #新しいパラメータの採択を決定
        rand = np.random.uniform(0, 1, 1)
        accept_prob[j] = np.min(np.append(1, np.exp(Hold - Hnew)))   #採択率

        #採択率に基づきgammaを採択
        if(accept_prob[j] >= rand):
            tau[j, ] = np.exp(tau_logn)
            tau1[j] = tau[j, 0]; tau2[j] = tau[j, 1]
        else:
            tau[j, ] = np.exp(tau_logd)
            tau1[j] = tau[j, 0]; tau2[j] = tau[j, 1]


    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[:, :, mkeep] = beta
        SIGMA[mkeep] = Sigma
        TAU[:, :, mkeep] = tau
        THETA[:, :, mkeep] = theta
        SEG[:, :, mkeep] = Zi

    if rp%disp==0:
        #対数尤度の和
        LL1 = np.sum(scipy.stats.norm.logpdf(y[:, k12], np.sum(X * beta.T[z_vec, ], axis=1), Sigma))
        LL2 = np.sum(scipy.stats.beta.logpdf(y[:, k11], tau1[z_vec], tau2[z_vec]))
        LL = LL1 + LL2

        #サンプリング結果の表示
        print(rp)
        print(np.round(np.array([LL, LL1, LL2, LLbest, LLbest1, LLbest2]), 3))
        print(np.round(accept_prob, 3))

0
[-3948.184 -3761.099  -187.086  2280.383 -2141.309  4421.692]
[1 1 0 1]
100
[-1771.585 -3437.154  1665.569  2280.383 -2141.309  4421.692]
[1 0 0 1]
200
[  -37.448 -3112.291  3074.843  2280.383 -2141.309  4421.692]
[1 1 0 0]
300
[  482.694 -3225.956  3708.65   2280.383 -2141.309  4421.692]
[1 1 0 0]
400
[  854.494 -3312.331  4166.825  2280.383 -2141.309  4421.692]
[1 1 1 0]
500
[ 1839.935 -2335.66   4175.595  2280.383 -2141.309  4421.692]
[1 1 1 1]
600
[ 1952.851 -2315.843  4268.694  2280.383 -2141.309  4421.692]
[0 0 1 1]
700
[ 1991.733 -2277.117  4268.85   2280.383 -2141.309  4421.692]
[0 1 1 0]
800
[ 1963.604 -2266.886  4230.49   2280.383 -2141.309  4421.692]
[1 0 1 0]
900
[ 1949.573 -2281.586  4231.159  2280.383 -2141.309  4421.692]
[0 0 1 0]
1000
[ 1951.275 -2259.466  4210.74   2280.383 -2141.309  4421.692]
[1 0 1 0]
1100
[ 1925.016 -2309.097  4234.113  2280.383 -2141.309  4421.692]
[1 1 0 1]
1200
[ 1946.16  -2275.016  4221.176  2280.383 -2141.309  4421.692]
[0 0 0 0]
1300
[ 1915